In [ ]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/AIM')
!ls -l /content/gdrive/MyDrive/AIM/*.jpg
import aim_util

Mounted at /content/gdrive
ls: cannot access '/content/gdrive/MyDrive/AIM/*.jpg': No such file or directory


In [ ]:
!ls -l /content/gdrive/MyDrive/AIM/*.jpg

-rw------- 1 root root 3663644 Apr 14 21:34 /content/gdrive/MyDrive/AIM/citizen_1864_20240414_213308040.jpg
-rw------- 1 root root 3663644 Apr 23 20:54 /content/gdrive/MyDrive/AIM/citizen_1864_rescan.jpg
-rw------- 1 root root 3890679 May  3 18:33 /content/gdrive/MyDrive/AIM/citizen_xform1.jpg
-rw------- 1 root root 4282849 May  3 18:33 /content/gdrive/MyDrive/AIM/citizen_xform2.jpg
-rw------- 1 root root 2669393 May  3 18:33 /content/gdrive/MyDrive/AIM/citizen_xform3.jpg
-rw------- 1 root root 6732009 May  3 18:33 /content/gdrive/MyDrive/AIM/citizen_xform4.jpg
-rw------- 1 root root 4093224 May  3 18:33 /content/gdrive/MyDrive/AIM/citizen_xform5.jpg
-rw------- 1 root root 2955757 May  6 20:57 /content/gdrive/MyDrive/AIM/deed_McGrew_to_HansOlson_1887_IMG_20191221_160034.jpg
-rw------- 1 root root 9444802 Mar 10 21:08 /content/gdrive/MyDrive/AIM/hans-olson-citizen-full-PXL_20240225_171151947.MP.jpg
-rw------- 1 root root 7437637 Mar 10 21:08 /content/gdrive/MyDrive/AIM/hans-olson-citize

In [ ]:
# Model A
import numpy as np
import json
from PIL import Image, ImageDraw, ImageFont

# Set the file paths
preds_path = '/content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv.json'
image_path = '/content/gdrive/MyDrive/AIM/citizen_1864_rescan.jpg'
save_path = '/content/gdrive/MyDrive/AIM/citizen_1864_overlay.jpg'

# # Load JSON data from the file
# with open(preds_path, 'r') as f:
#   predictions = json.load(f)

def json_to_preds(base_filename):
    json_filename = base_filename + ".json"
    with open(json_filename, "r") as json_file:
        json_data = json.load(json_file)
        #print(json_data)

    # Convert JSON lists back to numpy arrays with the original data type
    converted_preds = []
    for item in json_data:
      #print(item)
      converted_preds.append((
          item['key'],
          np.array(item['val'])
      ))
      #print(converted_preds)
    return converted_preds

# Test code
base_filename = '/content/gdrive/MyDrive/AIM/citizen_1864_xform0_carlv'
predictions = json_to_preds(base_filename)

# Filter out words with less than 3 characters
# filtered_predictions = [pred for pred in predictions if len(pred['word']) >= 3]
# print("Remaining words:")
# print([pred['word'] for pred in filtered_predictions])

# Load the image
img = Image.open(image_path)

# Draw the bounding boxes and words on the image
draw = ImageDraw.Draw(img)
font = ImageFont.truetype("/content/gdrive/MyDrive/AIM/OpenSans-Regular.ttf", 24)  # Adjust the font size as needed

for pred in predictions:
    x, y, w, h = pred['val']
    draw.rectangle([x, y, x+w, y+h], outline=(0, 255, 0))
    word_width, word_height = draw.textsize(pred['key'], font=font)

    # Adjust text position to ensure it fits within the bounding box
    text_x = max(x, x + (w - word_width) // 2)
    text_y = max(y, y + (h - word_height) // 2)
    draw.text((text_x, text_y), pred['key'], fill=(0, 0, 0), font=font)

# Save the modified image
img.save(save_path)

# Print the count of overlays
print(f"Count of bounding box/word overlays: {len(predictions)}")

TypeError: tuple indices must be integers or slices, not str

In [ ]:
# Model B